In [ ]:
!pip install -e /home/jovyan/notebooks/feast-spark

In [1]:
!feast apply

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/feast/importer.py", line 26, in import_class
    module = importlib.import_module(module_name)
  File "/opt/conda/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/jovyan/notebooks/feast-spark/feast_custom_offline_store/spark.py", line 33
    e

In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from sklearn.datasets import make_hastie_10_2
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def generate_entities(size):
    return np.random.choice(size, size=size, replace=False)

entities=generate_entities(101)

entity_df = pd.DataFrame(data=entities, columns=['entity_id'])
entity_df["event_timestamp"]=datetime(2021, 1, 3, 23, 59, 42, tzinfo=timezone.utc)
entity_df = entity_df[entity_df.entity_id==100]

entity_df

,entity_id,event_timestamp
71,100,2021-01-03 23:59:42+00:00


In [ ]:
from feast import FeatureStore
import pandas as pd

import time

store = FeatureStore(repo_path=".")

start_time = time.time()
training_df = store.get_historical_features(
    entity_df=pd.read_parquet('./dataset/entity_df.parquet'), 
    features = [
        'my_statistics:f0',
        'my_statistics:f1',
        'my_statistics:f2',
        'my_statistics:f3',
        'my_statistics:f4',
        'my_statistics:f5',
        'my_statistics:f6',
        'my_statistics:f7',
        'my_statistics:f8',
        'my_statistics:f9',
        'my_statistics:y',
    ],
).to_df()

print(training_df)
print("--- %s seconds ---" % (time.time() - start_time))